In [1]:
import pandas as pd
import numpy as np
from sklearn import svm

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [2]:
df = pd.read_excel('modified dataset for sentiment analysis.xlsx',header=0, delimiter="\t", quoting=3)
df.dropna(inplace=True)
df.head(10)

,teaching,Teaching_1,coursecontent,coursecontent.1,examination,Examination,labwork,labwork.1,library_facilities,library_facilities,extracurricular,extracurricular.1
0,-1,bad,0.0,content of courses are average,1.0,examination pattern is good,-1,"not satisfactory, lab work must include latest...",0.0,library facilities are good but number of book...,1,extracurricular activities are excellent and p...
1,1,Good,-1.0,Not good,1.0,Good,1,Good,-1.0,Not good,1,Good
2,1,Excellent lectures are delivered by teachers a...,1.0,All courses material provide very good knowled...,1.0,Exam pattern is up to the mark and the Cgpa de...,1,Lab work is properly covered in the labs by th...,1.0,Library facilities are excellent in terms of g...,1,Extra curricular activities also help students...
3,1,Good,-1.0,Content of course is perfectly in line with th...,-1.0,Again the university tests students of their a...,1,Good,0.0,Its the best thing i have seen in this univers...,-1,Complete wastage of time. Again this opinion i...
4,1,teachers give us all the information required ...,1.0,content of courses improves my knowledge,1.0,examination pattern is good,1,practical work provides detail knowledge of th...,1.0,library has huge collection of books from diff...,1,extracurricular activities increases mental an...
5,1,Yes,1.0,Yes,1.0,Yes,1,Yes,1.0,Yes,1,Yes
6,-1,our college aptitude class was very bored and ...,1.0,Good,1.0,Good,1,Good,1.0,Hardworking,0,No views
7,1,It is good,1.0,This semester university has provide best teac...,1.0,I like the question pattern,1,Everything is going fine in lab . learning new...,0.0,I am satisfied with the facilities but few boo...,0,No idea about the extracurricular activities.
8,0,waste of time,-1.0,Needs some improvement,1.0,Good,1,Good,-1.0,Its required some libral in rules,1,Good it provides a great platform
9,1,Good,-1.0,Needs some improvement,1.0,Good,1,Good,-1.0,Its required some libral in rules,1,Good it provides a great platform


In [3]:
from sklearn.model_selection import train_test_split
df["teaching_FB"]=np.where(df['teaching']>0,1,-1)
X_train, X_test, Y_train, Y_test = train_test_split(df['Teaching_1'],df['teaching'],random_state=0)

In [ ]:
#X = df.iloc[ : , 1:-10]
#Y = df.iloc[:, 0:1].values

In [ ]:
#from sklearn.cross_validation import train_test_split
#X_train, X_test, Y_train, Y_test = train_test_split( X , Y , test_size = 0.2, random_state = 0)


In [5]:
from collections import Counter

count_vocab = Counter()
for txt in X_train:
    for word in txt.split(' '):
        count_vocab[word] += 1
        
count_vocab.most_common(12)

[(u'good', 65),
 (u'is', 61),
 (u'the', 41),
 (u'and', 40),
 (u'of', 38),
 (u'are', 31),
 (u'', 30),
 (u'but', 24),
 (u'not', 21),
 (u'Good', 21),
 (u'in', 21),
 (u'delivery', 21)]

In [6]:
import nltk
nltk.download('stopwords')

/usr/local/lib/python2.7/dist-packages/sklearn/feature_extraction/hashing.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _hashing
/usr/local/lib/python2.7/dist-packages/sklearn/datasets/svmlight_format.py:25: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._svmlight_format import _load_svmlight_file


[nltk_data] Downloading package stopwords to /home/asus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

vocab_reduced = Counter()
for w, c in count_vocab.items():
    if not w in stop:
        vocab_reduced[w] = c
vocab_reduced.most_common(14)

[(u'good', 65),
 (u'', 30),
 (u'Good', 21),
 (u'delivery', 21),
 (u'teaching', 18),
 (u'lecture', 17),
 (u'university', 16),
 (u'teachers', 14),
 (u'students', 13),
 (u'interaction', 12),
 (u'terms', 11),
 (u'punctuality', 10),
 (u'faculty', 8),
 (u'teacher', 7)]

In [9]:
import re

def preprocessor(text):
    
    #remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    return text

print(preprocessor('This!! twit man :) is <b>nice</b>'))

this twit man is nice :)


In [10]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer('Hi there, I am loving this, like with a lot of love'))
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

['Hi', 'there,', 'I', 'am', 'loving', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']
['Hi', 'there,', 'I', 'am', u'love', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__C': [1.0, 10.0, 100.0]},
              ]
lr_tfidf = Pipeline([('vect', tfidf),('clf', LinearSVC(random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           verbose=1,
                           n_jobs=-1)

In [12]:
gs_lr_tfidf.fit(X_train, Y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:    3.8s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru..., max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__tokenizer': [<function tokenizer at 0x7f55861a7c80>, <function tokenizer_porter at 0x7f55861a7a28>], 'vect__preprocessor': [None, <function preprocessor at 0x7f55861a7aa0>], 'clf__C': [1.0, 10.0, 100.0], 'vect__stop_words': [[u'i', u'me', u'my', u'm...ction tokenizer_porter at 0x7f55861a7a28>], 'vect__use_idf': [False], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=True, return

In [13]:
print('Best parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Best accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer_porter at 0x7f55861a7a28>, 'vect__preprocessor': None, 'vect__stop_words': None, 'vect__norm': None, 'vect__use_idf': False, 'clf__C': 1.0}
Best accuracy: 0.793


In [14]:
clf = gs_lr_tfidf.best_estimator_
print('Accuracy in test: %.3f' % clf.score(X_test, Y_test))

Accuracy in test: 0.848


In [15]:
twits = []
user_feedback = raw_input("give ur feedback: ")
twits.append(user_feedback)

preds = clf.predict(twits)

for i in range(len(twits)):
    #print(preds[i], twits[i])
    if preds[i] == -1:
        print "Feedback Negative: {BAD}= ",(preds[i])
    elif preds[i] == 1:
        print "Feedback Positive: {GOOD}=",(preds[i])
    else:
        print "Feedback neutral: {AVERAGE}=",(preds[i])

give ur feedback: our college aptitude class was very bored and waste of time
Feedback Negative: {BAD}=  -1
